# Salinity

In [1]:
import xarray as xr
from pydap.client import open_url
import numpy as np

In [19]:
chunk_size = {'lat': 177, 'lon': 241, 'time': 500}

In [15]:
sea_salinity = xr.open_mfdataset('shared/data/surface_salinity/*.nc')

In [3]:
sea_salinity

<xarray.Dataset>
Dimensions:    (longitude: 721, latitude: 529, time: 10227)
Coordinates:
  * longitude  (longitude) float32 42.0 42.08 42.17 42.25 ... 101.8 101.9 102.0
  * latitude   (latitude) float32 -12.0 -11.92 -11.83 ... 31.83 31.92 32.0
  * time       (time) datetime64[ns] 1993-01-01T12:00:00 ... 2020-12-31T12:00:00
Data variables:
    so         (time, latitude, longitude) float32 dask.array<chunksize=(31, 529, 721), meta=np.ndarray>

In [4]:
ds = xr.open_zarr('shared/data/INDIAN_OCEAN_DAILY_025GRID.zarr/')

## TIME1

In [5]:
grid_time1 = ds.sel(time=slice('1979', '1992'))

In [6]:
grid_time1

<xarray.DataArray 'u_wind' (time: 5114, lat: 177, lon: 241)>
dask.array<getitem, shape=(5114, 177, 241), dtype=float32, chunksize=(100, 177, 241), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 32.0 31.75 31.5 31.25 ... -11.25 -11.5 -11.75 -12.0
  * lon      (lon) float32 42.0 42.25 42.5 42.75 ... 101.2 101.5 101.8 102.0
  * time     (time) datetime64[ns] 1979-01-01 1979-01-02 ... 1992-12-31
Attributes:
    long_name:       10 metre U wind component
    nameCDM:         10_metre_U_wind_component_surface
    nameECMWF:       10 metre U wind component
    product_type:    analysis
    shortNameECMWF:  10u
    standard_name:   eastward_wind
    units:           m s**-1

In [9]:
grid_time1.shape

(5114, 177, 241)

In [15]:
salinity_time1 = grid_time1.copy()

In [24]:
salinity_time2 = salinity_time1.where(~np.isnan(salinity_time1), np.nan)

In [30]:
salinity_time2.values[:] = np.nan

In [39]:
salinity_time2

<xarray.DataArray 'so' (time: 5114, lat: 177, lon: 241)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)
Coordinates:
  * lat      (lat) float32 32.0 31.75 31.5 31.25 ... -11.25 -11.5 -11.75 -12.0
  * lon      (lon) float32 42.0 42.25 42.5 42.75 ... 101.2 101.5 101.8 102.0
  * time     (time) datetime64[ns] 1979-01-01 1979-01-02 ... 1992-12-31

In [35]:
salinity_time2.name = 'so'

In [38]:
salinity_time2.attrs.clear()

In [41]:
salinity_time2 = salinity_time2.chunk(chunk_size)

In [42]:
salinity_time2

<xarray.DataArray 'so' (time: 5114, lat: 177, lon: 241)>
dask.array<xarray-<this-array>, shape=(5114, 177, 241), dtype=float32, chunksize=(100, 177, 241), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 32.0 31.75 31.5 31.25 ... -11.25 -11.5 -11.75 -12.0
  * lon      (lon) float32 42.0 42.25 42.5 42.75 ... 101.2 101.5 101.8 102.0
  * time     (time) datetime64[ns] 1979-01-01 1979-01-02 ... 1992-12-31

In [43]:
salinity_time2.to_netcdf('data/salinity_time1.nc')

## TIME2

In [52]:
grid_time2 = ds.sel(time=slice('1993', '1999'))
salinity_time2 = sea_salinity.sel(time=slice('1993', '1999'))['so']

In [53]:
salinity_time2 =salinity_time2.rename({'latitude': 'lat', 'longitude': 'lon'})

In [54]:
salinity_time2_interp = salinity_time2.interp_like(grid_time2)

In [57]:
salinity_time2_interp

<xarray.DataArray 'so' (time: 2556, lat: 177, lon: 241)>
dask.array<transpose, shape=(2556, 177, 241), dtype=float32, chunksize=(2556, 177, 241), chunktype=numpy.ndarray>
Coordinates:
  * lon      (lon) float32 42.0 42.25 42.5 42.75 ... 101.2 101.5 101.8 102.0
  * lat      (lat) float32 32.0 31.75 31.5 31.25 ... -11.25 -11.5 -11.75 -12.0
  * time     (time) datetime64[ns] 1993-01-01 1993-01-02 ... 1999-12-31

In [56]:
salinity_time2_interp.attrs.clear()

In [58]:
salinity_time2_interp.to_netcdf('data/salinity_time2.nc')

## TIME3

In [69]:
grid_time3 = ds.sel(time=slice('2000', '2009'))
salinity_time3 = sea_salinity.sel(time=slice('2000', '2009'))['so']
salinity_time3 =salinity_time3.rename({'latitude': 'lat', 'longitude': 'lon'})
salinity_time3_interp = salinity_time3.interp_like(grid_time3)
salinity_time3_interp.attrs.clear()

In [70]:
salinity_time3_interp

<xarray.DataArray 'so' (time: 3653, lat: 177, lon: 241)>
dask.array<transpose, shape=(3653, 177, 241), dtype=float32, chunksize=(3653, 177, 241), chunktype=numpy.ndarray>
Coordinates:
  * lon      (lon) float32 42.0 42.25 42.5 42.75 ... 101.2 101.5 101.8 102.0
  * lat      (lat) float32 32.0 31.75 31.5 31.25 ... -11.25 -11.5 -11.75 -12.0
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2009-12-31

In [71]:
salinity_time3_interp.to_netcdf('data/salinity_time3.nc')

## TIME4

In [72]:
grid_time4 = ds.sel(time=slice('2010', '2020'))
salinity_time4 = sea_salinity.sel(time=slice('2010', '2020'))['so']
salinity_time4 =salinity_time4.rename({'latitude': 'lat', 'longitude': 'lon'})
salinity_time4_interp = salinity_time4.interp_like(grid_time4)
salinity_time4_interp.attrs.clear()

In [73]:
salinity_time4_interp

<xarray.DataArray 'so' (time: 4018, lat: 177, lon: 241)>
dask.array<transpose, shape=(4018, 177, 241), dtype=float32, chunksize=(4018, 177, 241), chunktype=numpy.ndarray>
Coordinates:
  * lon      (lon) float32 42.0 42.25 42.5 42.75 ... 101.2 101.5 101.8 102.0
  * lat      (lat) float32 32.0 31.75 31.5 31.25 ... -11.25 -11.5 -11.75 -12.0
  * time     (time) datetime64[ns] 2010-01-01 2010-01-02 ... 2020-12-31

In [74]:
salinity_time4_interp.to_netcdf('data/salinity_time4.nc')

## TIME5

In [6]:
grid_time5 = ds['u_wind'].sel(time=slice('2021', '2022'))
salinity_time5=grid_time5.copy()
salinity_time5.name='so'
salinity_time5.attrs.clear()

In [7]:
salinity_time5.load()
salinity_time5.values[:] = np.nan

## Combine stuff

In [5]:
ds

<xarray.Dataset>
Dimensions:          (time: 16071, lat: 177, lon: 241)
Coordinates:
  * lat              (lat) float32 32.0 31.75 31.5 31.25 ... -11.5 -11.75 -12.0
  * lon              (lon) float32 42.0 42.25 42.5 42.75 ... 101.5 101.8 102.0
  * time             (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2022-12-31
Data variables: (12/15)
    CHL              (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    CHL_uncertainty  (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    adt              (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    air_temp         (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    direction        (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    sla              (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    ...               ...
    u_curr           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    u_wind           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    ug_curr          (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    v_curr           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    v_wind           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    vg_curr          (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
Attributes: (12/17)
    creator_email:              minhphan@uw.edu
    creator_name:               Minh Phan
    creator_type:               person
    date_created:               2023-07-07
    geospatial_lat_max:         32.0
    geospatial_lat_min:         -12.0
    ...                         ...
    geospatial_lon_units:       degrees_east
    source:                     Earth & Space Research (ESR), Copernicus Clim...
    summary:                    Daily mean of 0.25 x 0.25 degrees gridded dat...
    time_coverage_end:          2022-12-31T23:59:59
    time_coverage_start:        1979-01-01T00:00:00
    title:                      Climate Data for Coastal Upwelling Machine Le...

In [9]:
salinity_imported = xr.open_mfdataset('data/salinity_time*.nc')

In [10]:
salinity = xr.combine_by_coords([salinity_imported, salinity_time5])

In [13]:
salinity['so'].encoding

{'zlib': False,
 'szip': False,
 'zstd': False,
 'bzip2': False,
 'blosc': False,
 'shuffle': False,
 'complevel': 0,
 'fletcher32': False,
 'contiguous': True,
 'chunksizes': None,
 'source': '/home/jovyan/data/salinity_time1.nc',
 'original_shape': (5114, 177, 241),
 'dtype': dtype('float32'),
 '_FillValue': nan}

In [16]:
salinity['so'].attrs = sea_salinity['so'].attrs

In [17]:
ds['so'] = (['time', 'lat', 'lon'], salinity['so'].values)

In [20]:
ds = ds.chunk(chunk_size)

In [25]:
ds

<xarray.Dataset>
Dimensions:          (time: 16071, lat: 177, lon: 241)
Coordinates:
  * lat              (lat) float32 32.0 31.75 31.5 31.25 ... -11.5 -11.75 -12.0
  * lon              (lon) float32 42.0 42.25 42.5 42.75 ... 101.5 101.8 102.0
  * time             (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2022-12-31
Data variables: (12/16)
    CHL              (time, lat, lon) float32 dask.array<chunksize=(500, 177, 241), meta=np.ndarray>
    CHL_uncertainty  (time, lat, lon) float32 dask.array<chunksize=(500, 177, 241), meta=np.ndarray>
    adt              (time, lat, lon) float32 dask.array<chunksize=(500, 177, 241), meta=np.ndarray>
    air_temp         (time, lat, lon) float32 dask.array<chunksize=(500, 177, 241), meta=np.ndarray>
    direction        (time, lat, lon) float32 dask.array<chunksize=(500, 177, 241), meta=np.ndarray>
    sla              (time, lat, lon) float32 dask.array<chunksize=(500, 177, 241), meta=np.ndarray>
    ...               ...
    u_wind           (time, lat, lon) float32 dask.array<chunksize=(500, 177, 241), meta=np.ndarray>
    ug_curr          (time, lat, lon) float32 dask.array<chunksize=(500, 177, 241), meta=np.ndarray>
    v_curr           (time, lat, lon) float32 dask.array<chunksize=(500, 177, 241), meta=np.ndarray>
    v_wind           (time, lat, lon) float32 dask.array<chunksize=(500, 177, 241), meta=np.ndarray>
    vg_curr          (time, lat, lon) float32 dask.array<chunksize=(500, 177, 241), meta=np.ndarray>
    so               (time, lat, lon) float32 dask.array<chunksize=(500, 177, 241), meta=np.ndarray>
Attributes: (12/17)
    creator_email:              minhphan@uw.edu
    creator_name:               Minh Phan
    creator_type:               person
    date_created:               2023-07-07
    geospatial_lat_max:         32.0
    geospatial_lat_min:         -12.0
    ...                         ...
    geospatial_lon_units:       degrees_east
    source:                     Earth & Space Research (ESR), Copernicus Clim...
    summary:                    Daily mean of 0.25 x 0.25 degrees gridded dat...
    time_coverage_end:          2022-12-31T23:59:59
    time_coverage_start:        1979-01-01T00:00:00
    title:                      Climate Data for Coastal Upwelling Machine Le...

In [22]:
ds['so'].attrs = sea_salinity['so'].attrs

In [27]:
ds['so'].attrs

{'long_name': 'mean sea water salinity at 0.49 metres below ocean surface',
 'standard_name': 'sea_water_salinity',
 'units': '1e-3',
 'unit_long': 'Practical Salinity Unit',
 'valid_min': 1,
 'valid_max': 28336,
 'cell_methods': 'area: mean',
 '_ChunkSizes': array([  1,   7, 341, 720])}

In [26]:
ds['so'].attrs['long_name'] = 'mean sea water salinity at 0.49 metres below ocean surface'

In [28]:
ds.to_zarr('shared/data/FINAL_7_19_23.zarr/', consolidated=True)